In [18]:
import math

from qiskit import IBMQ, Aer, transpile, assemble
from qiskit import execute
from qiskit.circuit import QuantumCircuit
import qiskit.providers.aer.noise as noise
from qiskit.visualization import plot_histogram
from qiskit.circuit.library import MCMTVChain 
import csv


# Apply H-gates to put qubits in superstate
def initialize_s(qc, qubits):
    for q in qubits:
        qc.h(q)
    return qc

# Construct a diffuser for circuit of n qubits
def diffuser(nqubits):
    qc = QuantumCircuit(nqubits)
    for qubit in range(nqubits):
        qc.h(qubit)
    for qubit in range(nqubits):
        qc.x(qubit)
    qc.h(nqubits-1)
    qc.mct(list(range(nqubits-1)), nqubits-1)
    qc.h(nqubits-1)
    for qubit in range(nqubits):
        qc.x(qubit)
    for qubit in range(nqubits):
        qc.h(qubit)
    U_s = qc.to_gate()
    U_s.name = "Diffuser"
    return U_s


def execute_circ(n, noise_model=None, basis_gates=None, coupling_map=None):
    qs = list(range(n))
    grover_circuit = QuantumCircuit(n)
    grover_circuit = initialize_s(grover_circuit, qs)
    grover_circuit.append(diffuser(n), qs)
    grover_circuit.measure_all()
    grover_circuit.draw()

    n_seq = round(math.sqrt(math.pow(2,n)))
    result = execute(grover_circuit, Aer.get_backend('qasm_simulator'),
                     noise_model=noise_model,
                     coupling_map=coupling_map,
                     basis_gates=basis_gates,shots=100000).result()
    counts = result.get_counts()
    return counts

def custom_noise(prob, gates, error_function):
    custom_noise = noise.NoiseModel()
    e = error_function(prob, 1)
    custom_noise.add_all_qubit_quantum_error(e, gates)
    custom_basis = custom_noise.basis_gates
    return custom_noise, custom_basis

# Build noise model from backend properties
provider = IBMQ.load_account()
backend = provider.get_backend('ibmq_lima')

real_noise = noise.NoiseModel.from_backend(backend)
real_basis = real_noise.basis_gates
coupling_map = backend.configuration().coupling_map

# build custom noise models
std_err = 3e-3
depolar_h, depolar_basis_h = custom_noise(std_err, 'h', noise.depolarizing_error)
depolar_x, depolar_basis_x = custom_noise(std_err, 'x', noise.depolarizing_error)
ampdamp_h, ampdamp_basis_h = custom_noise(std_err, 'h', noise.amplitude_damping_error)
ampdamp_x, ampdamp_basis_x = custom_noise(std_err, 'x', noise.amplitude_damping_error)
phasedamp_h, phasedamp_basis_h = custom_noise(std_err, 'h', noise.phase_damping_error)
phasedamp_x, phasedamp_basis_x = custom_noise(std_err, 'x', noise.phase_damping_error)

bitcount = [2,3,5]
for x in bitcount:
    clean = execute_circ(x)
    real = execute_circ(x, real_noise, real_basis, coupling_map)
    depolar = execute_circ(x, depolar_h, depolar_basis_h)
    ampdamp = execute_circ(x, ampdamp_h, ampdamp_basis_h)
    phasedamp = execute_circ(x, phasedamp_h, phasedamp_basis_h)

    with open('test_'+ str(x)+ '.csv', 'w') as f:
        f.write("index, clean, real, depolar, ampdamp, phasedamp\n")
        for key in clean.keys():
            f.write("%s, %s, %s, %s, %s, %s\n" % (key, clean[key], real[key],depolar[key],ampdamp[key],phasedamp[key] ))
    
    legend = ['Without noise', 'With noise', 'Depolarizing', 'Amplification dampening', 'Phase dampening']
    plot_histogram([clean, real, depolar, ampdamp, phasedamp],
                   title=str(x) + " qubits",
                   legend=legend,
                   filename=f"{x}qubits.jpg",
                   figsize=(35,17))

    print(f"{x} qubits, clean: {clean}")
    print(f"{x} qubits, real: {real}")
    print(f"{x} qubits, depolarizing: {depolar}")
    print(f"{x} qubits, amplification dampening: {ampdamp}")
    print(f"{x} qubits, phase dampening: {phasedamp}")



ibmqfactory.load_account:WARNING:2022-05-10 13:56:30,633: Credentials are already in use. The existing account in the session will be replaced.


2 qubits, real: {'01': 25353, '10': 24672, '11': 23885, '00': 26090}
2 qubits, depolarizing: {'11': 25089, '01': 24793, '10': 25119, '00': 24999}
2 qubits, amplification dampening: {'10': 25121, '00': 25144, '01': 24831, '11': 24904}
2 qubits, phase dampening: {'01': 24771, '00': 24913, '11': 25336, '10': 24980}
3 qubits, real: {'001': 13686, '010': 13679, '100': 11660, '111': 10820, '110': 11363, '000': 14258, '011': 13377, '101': 11157}
3 qubits, depolarizing: {'111': 12484, '110': 12499, '000': 12493, '011': 12396, '010': 12665, '101': 12567, '001': 12394, '100': 12502}
3 qubits, amplification dampening: {'000': 12388, '001': 12453, '100': 12659, '111': 12621, '011': 12328, '110': 12562, '101': 12561, '010': 12428}
3 qubits, phase dampening: {'011': 12721, '110': 12661, '100': 12557, '010': 12363, '101': 12521, '111': 12263, '000': 12461, '001': 12453}
5 qubits, real: {'01001': 3152, '01110': 2895, '01000': 3548, '11001': 3048, '10001': 3385, '11110': 2778, '10101': 3132, '10110': 3